# Exploratory Data Analysis (EDA)

Steps:
1. Read dataset files
2. Dataset description
3. Get number of days
4. Get start and end dates
5. Identify Reversals
6. Price-Volume Correlation


# Step 1. Read dataset files

In [19]:
# Import file and data libraries
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from collections import Counter

In [20]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Read X & Y scaled datasets from Pickle files
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset.pickle", "rb") as f:
    X = pickle.load(f)
    Y = pickle.load(f) # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

# Step 2. Dataset Description

In [22]:
# Convert the Metadata into dataframe to use interactive table for analysis
Metadata_DF = pd.DataFrame(Metadata, columns = ["Company_ID", "Ticker", "Sector", "Dataset_ID"])
Metadata_DF

Company_ID Ticker Sector Dataset_ID
0             0   AAPL     IT          0
1             0   AAPL     IT          1
2             0   AAPL     IT          2
3             0   AAPL     IT          3
4             0   AAPL     IT          4
...         ...    ...    ...        ...
3984         19    NKE     CD        195
3985         19    NKE     CD        196
3986         19    NKE     CD        197
3987         19    NKE     CD        198
3988         19    NKE     CD        199

[3989 rows x 4 columns]

In [23]:
# Get the statistics of X
X_Stats = []
for i in range(len(X)):
    Len = len(X[i]["HL_Avg_Norm"])
    Min = np.min(X[i]["HL_Avg_Norm"])
    Max = np.max(X[i]["HL_Avg_Norm"])
    Mean = np.average(X[i]["HL_Avg_Norm"])
    Median = np.median(X[i]["HL_Avg_Norm"])
    Q1 = np.quantile(X[i]["HL_Avg_Norm"], 0.25)
    Q3 = np.quantile(X[i]["HL_Avg_Norm"], 0.75)
    Std = np.std(X[i]["HL_Avg_Norm"])
    Var = np.var(X[i]["HL_Avg_Norm"])
    X_Stats.append([Len, Min, Max, Mean, Median, Q1, Q3, Std, Var])
print(len(X_Stats))
print(X_Stats[0])

3989
[1264, 0.000997479771486573, 0.0032545309238579357, 0.0020030038123768504, 0.0019225015055134703, 0.0017004963722288208, 0.0022555095507813027, 0.00044227406295932096, 1.956063467665454e-07]


In [24]:
# Convert dataset Y into dataframe to use interactive table for analysis (to get the statistics)
DF_Y = pd.DataFrame(Y, columns=['Y_Avg', 'X_Last', 'Y_Diff', 'Y_Band', 'Y_Gap', 'Y_Days'])
DF_Y

Y_Avg     X_Last     Y_Diff Y_Band Y_Gap Y_Days
0      0.483371    0.47433   1.905844     U1    65      5
1      0.327232   0.379464  -13.76474     D3    66      5
2      0.423107   0.413125   2.416218     U1    64      5
3      0.414696   0.332946  24.553464     U5    63      5
4      1.098354   0.781529  40.539107     U5    66      4
...         ...        ...        ...    ...   ...    ...
3984     10.301   12.62625 -18.415998     D4    62      5
3985  32.434501       31.2   3.956733     U1    65      5
3986     22.898   23.86125  -4.036879     D1    64      5
3987    55.2875  61.575001 -10.211127     D3    66      4
3988   0.831836   0.611328  36.070342     U5    65      5

[3989 rows x 6 columns]

# Step 3. Get Number of Days

In [25]:
# Get the number of days
X_Len = []
for i in range(len(X)):
    X_Len.append(len(X[i]))
Counter(X_Len)

Counter({1230: 2,
         1231: 6,
         1232: 13,
         1233: 8,
         1234: 11,
         1235: 12,
         1242: 1,
         1243: 2,
         1247: 2,
         1249: 2,
         1254: 3,
         1255: 120,
         1256: 301,
         1257: 213,
         1258: 838,
         1259: 819,
         1260: 111,
         1261: 160,
         1262: 141,
         1263: 317,
         1264: 539,
         1265: 356,
         1266: 11,
         1267: 1})

# Step 4. Get Start and End Dates

In [26]:
# Get the start date and end date from the dataset X
Date_List = []
for ds in range(len(X)):
    Start_Date = datetime.strftime(X[ds]["Date"].iloc[0],"%Y-%m-%d")
    End_Date   = datetime.strftime(X[ds]["Date"].iloc[-1],"%Y-%m-%d")
    Days       = len(X[ds]["Date"])
    Date_List.append([Start_Date, End_Date, Days])

In [27]:
# Convert the arrow into dataframe to use interactive table for analysis
Date_DF = pd.DataFrame(Date_List, columns = ["Start_Date", "End_Date", "Days"])
Date_DF

Start_Date    End_Date  Days
0     1988-03-02  1993-03-01  1264
1     1982-07-28  1987-07-27  1264
2     1998-10-09  2003-10-08  1256
3     1996-10-24  2001-10-23  1257
4     1995-06-01  2000-05-31  1263
...          ...         ...   ...
3984  2003-11-26  2008-11-25  1259
3985  2008-06-03  2013-05-31  1258
3986  2007-08-10  2012-08-09  1261
3987  2011-03-30  2016-03-29  1257
3988  1984-06-05  1989-06-02  1263

[3989 rows x 3 columns]

# Step 5. Identify Reversals

In [ ]:
# Identify reversals in all datasets
# Run duration : 7 minutes
No_Of_Reversals_List = [] # For storing the number of reversals in each dataset

for DS in range(0, len(X)): 
    
    MA20_List = []
    Diff_List = []
    Reversal_List = []

    for Row in range (0, len(X[DS]["HL_Avg"])):

        # Calculate the 20-days Moving Average 
        # It starts at row 20, averaging row 0 to 19 (20 rows)
        if Row < 19: MA20 = 0 # In 20-days Moving Average, the first 20 rows are blank
        else: MA20 = X[DS]["HL_Avg"].iloc[Row-19:Row+1].mean() # The 20 days MA starts at row 20
        MA20_List.append(MA20)

        # Calculate the difference between the current row and 19 rows before it
        # It starts at row 38, which is row 38 - row 19 (the index starts at 0)
        if Row < 38: Diff = 0 # The first 38 rows are blank (index starts at 0)
        else: Diff = MA20 - MA20_List[Row-19] # Starts at row 38
        Diff_List.append(Diff)

        # Identify reversal based on the flipping of sign
        if Row < 39: Reversal = 0 # The reversal can't be calculated for the first 39 rows (index starts at 0)
        else: 
            if np.sign(Diff) == np.sign(Diff_List[Row-1]): Reversal = 0 # If the sign is the same, there is no reversal
            else: Reversal = 1 # if the sign flips, it's a reversal
        Reversal_List.append(Reversal)
        
    No_Of_Reversals = sum(Reversal_List)
    print(DS, No_Of_Reversals)
    No_Of_Reversals_List.append([DS, No_Of_Reversals])

In [29]:
# Convert the reversal array into Pandas dataframe to use interactive table for analysis
No_Of_Reversals_DS = pd.DataFrame(No_Of_Reversals_List, columns = ["Dataset","Reversals"])
No_Of_Reversals_DS

Dataset  Reversals
0         0         32
1         1         26
2         2         29
3         3         28
4         4         27
..      ...        ...
95       95         27
96       96         29
97       97         33
98       98         34
99       99         30

[100 rows x 2 columns]

# Step 6. Price-Volume Correlation

In [30]:
# Get the Pearson correlation coefficient between the price and volume for each dataset
Corr_List = []
for ds in range(len(X)):
    Corr = X[ds][["HL_Avg","Volume"]].corr(method='pearson')["HL_Avg"][1]
    Corr_List.append(Corr)

In [31]:
# Convert the correlation array into Pandas dataframe to use interactive table for analysis
Corr_DF = pd.DataFrame(Corr_List, columns = ["Corr"])
Corr_DF

Corr
0     0.064850
1     0.114738
2     0.046887
3     0.080171
4     0.199093
...        ...
3984  0.157233
3985 -0.367595
3986 -0.285403
3987 -0.189017
3988  0.289501

[3989 rows x 1 columns]